In [ ]:
import json

# 파일 이름
input_filename = 'keras-team_keras_issues_4875_class.json'
output_filename = 'sorted_keras-team_keras_issues_4875_class.json'

# JSON 파일 읽기
with open(input_filename, 'r', encoding='utf-8') as file:
    data = json.load(file)

# issueObjectAnchor 기준으로 정렬하는 함수
def sort_by_issue_object_anchor(data):
    return sorted(data, key=lambda x: int(x["issueObjectAnchor"].replace("IssueSPTOText", "")))

# 정렬 수행
sorted_data = sort_by_issue_object_anchor(data)

# 정렬된 데이터를 새로운 파일에 저장
with open(output_filename, 'w', encoding='utf-8') as file:
    json.dump(sorted_data, file, ensure_ascii=False, indent=4)

print(f"정렬된 데이터가 '{output_filename}'에 저장되었습니다.")

In [ ]:
# 파일 이름
issues_filename = 'sorted_keras-team_keras_issues_4875_class.json'
actions_filename = 'keras-team_keras_issues_4875_Start_End.json'
output_filename = 'merged_issues.json'

# JSON 파일 읽기
with open(issues_filename, 'r', encoding='utf-8') as file:
    issues = json.load(file)

with open(actions_filename, 'r', encoding='utf-8') as file:
    actions = json.load(file)

# 합칠 데이터 생성
merged_data = []

# 각 이슈에 대해 합치기
for issue in issues:
    issue_index = int(issue["issueObjectAnchor"].replace("IssueSPTOText", ""))
    
    # actions 리스트에서 start와 end 범위에 맞는 액션 찾기
    for action in actions:
        if action["start"] <= issue_index < action["end"]:
            merged_issue = {**issue, **action}  # issue와 action을 병합
            merged_data.append(merged_issue)
            break

# 결과를 새로운 파일에 저장
with open(output_filename, 'w', encoding='utf-8') as file:
    json.dump(merged_data, file, ensure_ascii=False, indent=4)

print(f"합쳐진 데이터가 '{output_filename}'에 저장되었습니다.")

In [ ]:
import pandas as pd

df = pd.read_json('merged_issues.json')

df

In [ ]:
import json
import itertools
import os

# JSON 파일 읽기
with open('merged_issues.json') as f:
    data = json.load(f)

# textType 목록
text_types = [
    'Action on Issue',
    'Bug Reproduction',
    'Contribution and Commitment',
    'Expected Behaviour',
    'Investigation and Exploration',
    'Motivation',
    'Observed Bug Behaviour',
    'Potential New Issues and Requests',
    'Social Conversation',
    'Solution Discussion',
    'Solution Usage',
    'Task Progress',
    'Workarounds'
]

# 비율 목록
ratios = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]

# 결과를 저장할 폴더 생성
output_dir = 'filtered_results'
os.makedirs(output_dir, exist_ok=True)

# 모든 조합 생성
for r in range(len(text_types) + 1):
    for hidden_types in itertools.combinations(text_types, r):
        for ratio_values in itertools.product(ratios, repeat=len(text_types)):
            # 필터링 조건 설정
            hidden_set = set(hidden_types)
            filtered_data = []

            for entry in data:
                total_line = entry['end'] - entry['start']

                # 비율 체크
                should_remove = False
                for text_type, ratio in zip(text_types, ratio_values):
                    if text_type in hidden_set:
                        continue
                    if entry[text_type.replace(' ', '')] / total_line > ratio:
                        should_remove = True
                        break

                if not should_remove:
                    filtered_data.append(entry)

            # 파일 이름 생성
            hidden_types_str = "_".join(hidden_set).replace(' ', '_')
            ratio_str = "_".join(map(str, ratio_values))
            file_name = f"필터됨_{hidden_types_str}_{ratio_str}.json"

            # 결과를 JSON 파일로 저장
            with open(os.path.join(output_dir, file_name), 'w', encoding='utf-8') as out_file:
                json.dump(filtered_data, out_file, ensure_ascii=False, indent=4)
